#### AFSK Demodulator
## Step 6: HDLC

This is a Pynq portion of the AFSK demodulator project.  We will be using the FPGA overlay that we created in Vivado.

At this point we have created the bitstream for "project_06" and copied the bitstream, TCL wrapper, and hardware hand-off file to the Pynq board.

Let's first verify that we can load the module.

In [1]:
import sys
sys.path.append('../../base')

from pynq import Overlay, Xlnk
import pynq.lib.dma

overlay = Overlay('project_10.bit')
dma = overlay.demodulator.dma
demod = overlay.demodulator.demod
print(demod)
print(demod.register_map)
print(demod.register_map.cancel_o)

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED=0),
  cancel_i = Register(cancel_i=0),
  cancel_o = Register(cancel_o=0),
  cancel_o_ctrl = Register(cancel_o_ap_vld=0, RESERVED=0)
}
0x0


## Implementing HDLC

Below is the implementation of the AFSK demodulator in Python.  We are now going to remove the HDLC code from our existing project.  We need to make a significant change to the Python code here.  We are no longer writing X items and reading back X items.  We are now feeding in audio data and will occassionally get back a packet.  The packet code will need to be dealt with asynchronously.

Luckily, the Pynq environment fully supports Python's asyncio interface for DMA operations.  When the packet arrive, we will decode the AX.25 data and print it, just like before.

In [3]:
import numpy as np
from scipy.io.wavfile import read
from AX25 import AX25
import time
import asyncio
import concurrent.futures

block_size = 1024*1024

xlnk = Xlnk()

async def read_packet():
    count = 0
    with xlnk.cma_array(shape=(4096,), dtype=np.uint8) as in_buffer:
        while True:
            dma.recvchannel.transfer(in_buffer)
            await dma.recvchannel.wait_async()
            in_buffer.invalidate();
            received = dma.recvchannel._mmio.read(dma.recvchannel._offset + 0x28)
            # print("received", received, in_buffer[received - 1])
            if in_buffer[received - 1] & 2:
                print("Done Reading", received, in_buffer[received - 1])
                print(demod.register_map.cancel_o)
                demod.register_map.cancel_i = 0
                demod.register_map.CTRL = 0x01 # START ONE to cancel cancellation
                break
            if not (received > 12 and in_buffer[received - 1] & 1):
                # print("Bad packet", in_buffer[received - 1])
                continue
            else:
                count += 1
            packet = ''.join(str(s, encoding='Latin-1') for s in in_buffer[:received-3])
            try:
                print(count, AX25(packet), received)
                # sys.stdout.write('\r%05d' % count)
            except:
                print("decode error: ", packet, received)

async def write_audio(data):
    demod.register_map.CTRL = 0x81 # START|AUTO-RESTART while writing data
    with xlnk.cma_array(shape=(block_size,), dtype=np.int16) as out_buffer:
        for i in range(0, len(data), block_size):
            size = len(data[i:i+block_size])
            if size != block_size: break
            out_buffer[:] = data[i:i+block_size]
            out_buffer.flush();
            dma.sendchannel.transfer(out_buffer)
            await dma.sendchannel.wait_async()
    
    with xlnk.cma_array(shape=(size,), dtype=np.int16) as out_buffer:
        out_buffer[:] = data[i:i+size]
        dma.sendchannel.transfer(out_buffer)
        await dma.sendchannel.wait_async()

    print("Done writing")
    demod.register_map.CTRL = 0x00 # STOP
    demod.register_map.cancel_i = 1
    demod.register_map.CTRL = 0x01 # START ONE for cancellation

def run():
    audio_file = read('../../base/TNC_Test_Ver-1.101-26400-1min.wav')
    sample_rate = audio_file[0]
    audio_data = np.append(audio_file[1], np.zeros(120, dtype=int))

    print("Starting...")
    start_time = time.time()

    # demodulate the audio data
    loop = asyncio.get_event_loop()
    loop.run_until_complete(asyncio.gather(
        read_packet(),
        write_audio(audio_data)
    ))
    stop_time = time.time()
    sw_exec_time = stop_time - start_time
    print('Total execution time:', sw_exec_time)

run()

Starting...
1 WA6YLB-8>APRS,N6EX-11:$ULTW00000000----0000----000086A00001----0000000000000000?? 85
2 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 78
3 KD6FVP-4>APS224,N6EX-2,WIDE1-1:>152343z[224]*We know most of your faults!!!? 78
4 N6XQY-8>GPSLJ,RELAY,WIDE2-5:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73? 104
5 WA6YLB>APRX46,WA6YLB-14,W6SCE-5:>081839z wa6ylb@theworks.com? 62
6 KC6HUR-2>S4QVYV,W6SCE-5:'.4&l-/k/]"7q}? 41
7 N6XQY-8>GPSLJ,N6EX-9:$GPRMC,013641.06,A,3348.1607,N,11807.4631,W,34.0,090.5,231105,13.,E*73? 97
8 KC6BLF-12>S4PWYS,N6EX-11:'-U l{(u/]"5\}Lost in the West!? 58
9 K6KMA-2>GPSLK,N6EX-3:$GPRMC,013647,A,3350.076,N,11806.996,W,028.3,180.5,231105,013.5,E*69?? 96
10 AE6GR-14>S4PXYW,WIDE2-5:'._|l tv/]"6[}? 41
11 AE6GR-14>S4PXYW,N6EX-3:'._|l tv/]"6[}? 41
12 AE6MP>SS5PPQ-4,WIDE2-5:`.](n->>/"4W} 39
13 AE6MP>SS5PPQ-4,N6EX-9:`.](n->>/"4W} 39
14 AE6MP>SS5PPQ-4,N6EX-3:`.](n->>/"4W} 39
15 AE6MP>SS5PPQ-2,N6EX-11:`.](n->>/"4W} 39


In [ ]:
xlnk.xlnk_reset()

In [ ]:
ctrl = dma.recvchannel._mmio.read(dma.recvchannel._offset)
print(ctrl)
dma.recvchannel._mmio.write(dma.recvchannel._offset, (ctrl | 4) & 0xFFFFFFFE)
print(dma.recvchannel._mmio.read(dma.recvchannel._offset+0x04))
dma.recvchannel.start()
dma.sendchannel.start()

In [ ]:
dma.recvchannel.stop()
dma.recvchannel.start()
with xlnk.cma_array(shape=(4096,), dtype=np.uint8) as in_buffer:
    while True:
        start_time = time.time()
        dma.recvchannel.transfer(in_buffer)
        dma.recvchannel.wait()
        dma.recvchannel.stop()
        dma.recvchannel.start()
        received = dma.recvchannel._mmio.read(dma.recvchannel._offset + 0x28)
        print("Received: ", received)
        packet = ''.join(str(s, encoding='Latin-1') for s in in_buffer[:received])
        print(AX25(packet))
        stop_time = time.time()
        sw_exec_time = stop_time - start_time
        print('FPGA read_packet execution time: ',sw_exec_time)


In [ ]:
demod.register_map.CTRL = 0

In [ ]:
print(demod.register_map)

In [ ]:
demod?

In [ ]:
print(demod.register_map.CTRL.AUTO_RESTART)